동적 크롤링
  - selenium
  - Web Driver 가 필요

In [ ]:
# 셀리니움 패키지 필요
!pip install selenium

In [ ]:
# 코렙에서 크롬웹드라이버를 사용할수 있도록 다운로드 - 자기컴퓨터의 로컬환경에서는 불필요
!apt-get update  # apt 업데이트
!apt install chromium-chromedriver # 크롬드라이버 설치
!cp /usr/lib/chromium-browser/chromedriver /usr/bin  # 크롬 드라이버를  해당경로에 복사

In [67]:
# 크롬 드라이버 생성
from selenium import webdriver
# 옵션 설정 --- 코렙에서는 아래와 같은 옵션이 없으면 보안상 불허.  실제동작하는 또하나의 웹 브라우져를 hidden
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# 크롭 드라이버 객체 생성
wd = webdriver.Chrome(options=options)

동적 크롤링

In [9]:
# 동적 크롤링할 웹 페이지 연결
wd.get("https://www.coffeebeankorea.com/store/store.asp")  # 로컬 가상환경에서 실행하면 자동으로 제어되는 브라우져가 새롭게 생성되면서 실행모습이 보임

In [10]:
# 필요한 자바스크립트 실행
wd.execute_script("storePop2(389)")

In [11]:
# 자바스크립트가 실행한 페이지소스코드 저장
html = wd.page_source

정적크롤링과 동일한 절차

In [17]:
# BeautifulSoup 객체 생성
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')

In [33]:
store_name =  soup.select('#matizCoverLayer0Content > div > div > div.store_txt > h2')[0]
store_info = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody:nth-child(1)>tr')
data_list = [store_name.text]
for tag in store_info:
  data_list.append(tag.find('td').text)

In [34]:
import pandas as pd
df = pd.DataFrame(data_list).T
df.columns = ['지점명','영업시간','주차','주소','전화번호']
df

,지점명,영업시간,주차,주소,전화번호
0,삼성본관빌딩점,월-금 07:00~19:00 | 토/일/공휴일 휴점,,서울특별시 중구 세종대로 67 삼성 본관빌딩 1층 후문 로비 일부,02-720-5561


함수로 제작

In [62]:
import time # delay를 발생해서 웹페이가 로드되는 시간을 기다린다.
def getStore(storeNum):
  wd.get("https://www.coffeebeankorea.com/store/store.asp")
  time.sleep(1) # 1초 동안 대기
  try:
    wd.execute_script(f"storePop2({storeNum})")
    time.sleep(1) # 1초 동안 대기
    html = wd.page_source
    soup = BeautifulSoup(html,'html.parser')
    store_name =  soup.select('#matizCoverLayer0Content > div > div > div.store_txt > h2')[0]
    store_info = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody:nth-child(1)>tr')
    data_list = [store_name.text]
    for tag in store_info:
      data_list.append(tag.find('td').text)
    df = pd.DataFrame(data_list).T
    df.columns = ['지점명','영업시간','주차','주소','전화번호']
    return df
  except:
    return None

In [68]:
from tqdm import tqdm
store_ds = []
for snum in tqdm(range(350,400)):
  result = getStore(snum)
  if type(result) == pd.DataFrame:
    store_ds.append(result)

100%|██████████| 50/50 [05:25<00:00,  6.51s/it]


In [71]:
df = pd.concat(store_ds)
df = df.reset_index(drop=True)
df

,지점명,영업시간,주차,주소,전화번호
0,하나은행을지로본점,| 월-금 07:00~19:00 | 주말/공휴일 휴무,,서울시 중구 을지로 35 하나은행 을지로본점 24F 내 스마트워크센터,02-338-9179
1,선릉KSA한국표준협회점,월-금 07:00~21:00 | 토요일 07:00~21:00 | 일요일 08:00~...,"건물 기계식 지하 주차 이용 (30분 무료 주차 후 30분당 3,000원)",서울시 강남구 테헤란로69길 5 DT센터점 1층,02-3141-7948
2,수서역원방빌딩점,평일 06:30 ~ 21:30 | 주말 07:00 ~ 21:30,,서울 강남구 광평로 51길 6-23 원방테크사옥 1층,02-3473-3830
3,서초중앙로점,평일 07:00~22:00 | 토-일 07:30~22:00 I (31~1일) 08:...,주차 불가,"서울시 서초구 서초중앙로 209, 해성빌딩 1층",02-537-2992
4,혜화역점,매일 08:00 ~ 00:00,,서울시 종로구 대명길 5,02-745-9471
5,진주LH본사점,월~금 08:00 ~ 18:00 | 토 10:00 ~ 18:00 | 일 휴무,,경남 진주시 충의로 19 LH진주사옥 공감동 1층,055-763-3520
6,하나금융명동사옥점,"월~금 06:30 ~ 22:00 | 토,일 08:00 ~ 22:00",,서울시 중구 을지로 66 하나은행 명동사옥 1층,02-539-5174
7,성균관대법학관점,월~금 08:00 ~ 20:00 | 토 09:00 ~ 17:00 | 일 휴점,,서울시 종로구 성균관로 25-2 성균관대학교 법학관 B2,02-740-1958
8,서울시청점,월-일 07:00 ~ 22:00,,서울특별시 중구 무교로 12(무교동) 정오빌딩 1층,02-420-0386
9,종로YBM별관점,"| 월~금 07:30~22:00 l 주말,공휴일 08:00~22:00 l",,서울시 종로구 종로 98 1층,02-402-3571
